In [1]:
def leggi_contenuto(file): #la funzione restituisce in output una variabile che ha memorizzato i contenuti del file con il metodo .read()
    with open(file, "r") as infile:
        contenuto = infile.read() #il metodo .read() consente di memorizzare il contenuto del file in una variabile
    return contenuto

file1 = "./janeausten.txt"
corpus1 = leggi_contenuto(file1)

print(f"Analisi di {file1}")

Analisi di ./janeausten.txt


In [2]:
import nltk

def splitter_e_tokenizzatore(corpus): #la funzione effettua il sentence splitting e la tokenizzazione del corpus
    frasi = nltk.tokenize.sent_tokenize(corpus) #splitta il testo in frasi
    tokens = []
    for frase in frasi:
        tokens_frase = nltk.tokenize.word_tokenize(frase) #tokenizza ciascuna frase
        tokens.extend(tokens_frase) # .extend() aggiunge in coda alla lista di tokens del corpus i tokens della frase
    return frasi, tokens

def PoS_tagger(tokens): #la funzione effettua il PoS tagging data la lista di tokens del corpus
    tokens_PoS = nltk.tag.pos_tag(tokens)
    return tokens_PoS #restituisce la lista di tuple (token,PoS)

frasi1, tokens1 = splitter_e_tokenizzatore(corpus1)
tokens_PoS1 = PoS_tagger(tokens1)

In [3]:
from nltk import FreqDist

def stampa_lista_di_tuple(lista): #funzione ausiliaria che formatta la stampa in colonna di una lista di tuple costituite da 2 elementi
    for i in range(len(lista)): #range(len(list)) crea una sequenza di numeri da 0 a len(list)-1
        print(f"{lista[i][0]}".ljust(50,"_")+f"{lista[i][1]}") #il metodo .ljust() consente di giustificare a sinistra una stringa usando "_" come separatore

def trova_ngrammiPoS_frequenti(tokens,n): #la funzione estrae i 10 n-grammi di PoS più frequenti
    PoS_tags = [PoS for token,PoS in PoS_tagger(tokens)] #mediante la sintassi di list comprehension per ogni tupla (token,PoS) restituita dalla funzione "PoS_tagger" vengono estratti solo i PoS tags
    if n==1: #se n=1 (unigrammi) calcola e ordina direttamente le frequenze dei tags in PoS_tags
        ngrammi_PoS_frequenti = nltk.FreqDist(PoS_tags).most_common(10) #con "FreqDist" (funzione di libreria) si ottiene un dizionario di frequenze ordinato in maniera decrescente; con .most_common(10) si ottiene una lista dei primi 10 elementi
    else: #se n!=1 (n-grammi) estrae prima gli n-grammi e poi calcola e ordina le frequenze
        ngrammi_PoS = list(nltk.ngrams(PoS_tags, n)) #estrae n-grammi con il metodo nltk.ngrams()
        ngrammi_PoS_frequenti = nltk.FreqDist(ngrammi_PoS).most_common(10)
    return ngrammi_PoS_frequenti #restituisce la lista di 10 tuple (ngramma,frequenza) ordinate per frequenza decrescente

print("10 PoS più frequenti (con relativa frequenza)")
stampa_lista_di_tuple(trova_ngrammiPoS_frequenti(tokens1,1))
print("")
print("10 bigrammi di PoS più frequenti (con relativa frequenza)")
stampa_lista_di_tuple(trova_ngrammiPoS_frequenti(tokens1,2))
print("")
print("10 trigrammi di PoS più frequenti (con relativa frequenza)")
stampa_lista_di_tuple(trova_ngrammiPoS_frequenti(tokens1,3))

10 PoS più frequenti (con relativa frequenza)
NN________________________________________________763
IN________________________________________________662
,_________________________________________________500
DT________________________________________________495
PRP_______________________________________________454
RB________________________________________________437
JJ________________________________________________404
NNP_______________________________________________354
VBD_______________________________________________337
CC________________________________________________291

10 bigrammi di PoS più frequenti (con relativa frequenza)
('DT', 'NN')______________________________________252
('NN', 'IN')______________________________________231
('NN', ',')_______________________________________180
('IN', 'DT')______________________________________169
('JJ', 'NN')______________________________________163
('PRP', 'VBD')____________________________________130
('NNP', 'NNP')_________________

In [4]:
def trova_classePoS_frequenti(tokens_PoS,filtro_PoS): #la funzione estrae e stampa i 20 token più frequenti appartenenti a una certa classe di PoS dati la lista (token,PoS) e un filtro PoS
    tokens_filtrati = [token for token,PoS in tokens_PoS if PoS.startswith(filtro_PoS)] #mediante la sintassi di list comprehension sono estratti i token appartenenti alla stessa classe di PoS filtrata con il metodo .startswith() perché a una PoS (es. Sostantivo) possono corrispondere più tags (es. "NN", "NNP", "NNPS", "NNS")
    tokens_frequenti = nltk.FreqDist(tokens_filtrati).most_common(20) #con "FreqDist" (funzione di libreria) si ottiene un dizionario di frequenze ordinato in maniera decrescente; con .most_common(20) si ottiene una lista dei primi 20 elementi
    return tokens_frequenti #restituisce la lista di 20 tuple (token,frequenza) ordinate per frequenza decrescente

print("20 Sostantivi più frequenti (con relativa frequenza)")
stampa_lista_di_tuple(trova_classePoS_frequenti(tokens_PoS1,"N"))
print("")
print("20 Avverbi più frequenti (con relativa frequenza)")
stampa_lista_di_tuple(trova_classePoS_frequenti(tokens_PoS1,"R"))
print("")
print("20 Aggettivi più frequenti (con relativa frequenza)")
stampa_lista_di_tuple(trova_classePoS_frequenti(tokens_PoS1,"J"))

20 Sostantivi più frequenti (con relativa frequenza)
Mr._______________________________________________37
Miss______________________________________________25
father____________________________________________23
Taylor____________________________________________23
Emma______________________________________________21
Elliot____________________________________________21
Walter____________________________________________20
years_____________________________________________18
Sir_______________________________________________17
Weston____________________________________________14
man_______________________________________________14
Knightley_________________________________________13
Elizabeth_________________________________________13
family____________________________________________11
friend____________________________________________11
Lady______________________________________________10
Woodhouse_________________________________________9
house__________________________________________

In [5]:
import math

def estrai_bigrammi_agg_sost(tokens): #la funzione estrae i bigrammi composti da aggettivo e sostantivo
    bigrammi_token_PoS = list(nltk.bigrams(PoS_tagger(tokens))) #sono estratti i bigrammi di tuple (token,pos) ottenute dal PoS tagging effettuato sui tokens in input    
    bigrammi_agg_sost = [(token1,token2) for (token1,PoS1),(token2,PoS2) in bigrammi_token_PoS if PoS1.startswith("J") and PoS2.startswith("N")] #mediante la sintassi di list comprehension sono estratti i bigrammi (token_aggettivo,token_sostantivo); è usato il metodo .startswith() perché a una PoS (es. Sostantivo) possono corrispondere più tags (es. "NN", "NNP", "NNPS", "NNS")
    return bigrammi_agg_sost #restituisce la lista di tuple (token,token) di cui il primo è un aggettivo e il secondo è un sostantivo

def trova_bigrammi_frequenti(bigrammi): #la funzione restituisce i bigrammi più frequenti data una lista di bigrammi
    bigrammi_frequenti = nltk.FreqDist(bigrammi).most_common(20) #con "FreqDist" (funzione di libreria) si ottiene un dizionario di frequenze ordinato in maniera decrescente; con .most_common(20) si ottiene una lista dei primi 20 elementi
    return bigrammi_frequenti #restituisce la lista delle prime 20 tuple (bigramma,frequenza) ordinate per frequenza decrescente

print("I 20 bigrammi Aggettivo, Sostantivo più frequenti (con relativa frequenza)")
stampa_lista_di_tuple(trova_bigrammi_frequenti(estrai_bigrammi_agg_sost(tokens1)))

I 20 bigrammi Aggettivo, Sostantivo più frequenti (con relativa frequenza)
('poor', 'Miss')__________________________________4
('young', 'man')__________________________________4
('lucky', 'guess')________________________________3
('excellent', 'woman')____________________________2
('other', 'people')_______________________________2
('great', 'deal')_________________________________2
('odd', 'humours')________________________________2
('intimate', 'friend')____________________________2
('great', 'regard')_______________________________2
('more', 'matches')_______________________________2
('such', 'success')_______________________________2
('good', 'looks')_________________________________2
('thirteen', 'years')_____________________________2
('comfortable', 'home')___________________________1
('happy', 'disposition')__________________________1
('best', 'blessings')_____________________________1
('indulgent', 'father')___________________________1
('early', 'period')______________________

In [6]:
def trova_bigrammi_probCondizionata_massima(bigrammi,tokens): #la funzione restituisce i bigrammi con Probabilità Condizionata massima data la lista di bigrammi e la lista di tokens
    bigrammi_frequenza = nltk.FreqDist(bigrammi).most_common() #il metodo .most_common() privo di argomento restituisce la lista completa di tuple (bigramma,frequenza)
    bigrammi_probabilita = {} #crea un dizionario per memorizzare la probabilità di ciascun bigramma
    for bigramma,frequenza in bigrammi_frequenza: #si scorre la lista "bigrammi_frequenza"
        frequenza_parola1 = tokens.count(bigramma[0]) #si ottiene la frequenza della prima parola del bigramma (che è una tupla (token,token))
        probabilita_bigramma = frequenza/frequenza_parola1 #calcola la probabilità condizionata del bigramma
        bigrammi_probabilita.update({bigramma:probabilita_bigramma}) #aggiorna il dizionario con il nuovo valore {bigramma:probabilità}
    bigrammi_probabilita_decrescente = sorted(bigrammi_probabilita.items(), key=lambda x:x[1], reverse=True) #a partire dal dizionario di ottiene una lista ordinata per valori decrescenti con la funzione di libreria "sorted"
    return bigrammi_probabilita_decrescente[:20] #si stampano i primi 20 elementi della lista con lo slicing

print("I 20 bigrammi Aggettivo, Sostantivo con probabilità condizionata massima (e relativo valore)")
stampa_lista_di_tuple(trova_bigrammi_probCondizionata_massima(estrai_bigrammi_agg_sost(tokens1),tokens1))

I 20 bigrammi Aggettivo, Sostantivo con probabilità condizionata massima (e relativo valore)
('odd', 'humours')________________________________1.0
('indulgent', 'father')___________________________1.0
('indistinct', 'remembrance')_____________________1.0
('Sixteen', 'years')______________________________1.0
('nominal', 'office')_____________________________1.0
('mournful', 'thought')___________________________1.0
('unexceptionable', 'character')__________________1.0
('easy', 'fortune')_______________________________1.0
('suitable', 'age')_______________________________1.0
('generous', 'friendship')________________________1.0
('past', 'kindness')______________________________1.0
('various', 'illnesses')__________________________1.0
('tenderer', 'recollection')______________________1.0
('intellectual', 'solitude')______________________1.0
('actual', 'disparity')___________________________1.0
('older', 'man')__________________________________1.0
('daily', 'reach')_________________________

In [7]:
def trova_bigrammi_PMI_massima(bigrammi,tokens): #la funzione restituisce i bigrammi con Pointwise Mutual Information massima data la lista di bigrammi e la lista di tokens
    bigrammi_frequenza = nltk.FreqDist(bigrammi).most_common() #il metodo .most_common() privo di argomento restituisce la lista completa di tuple (bigramma,frequenza)
    bigrammi_PMI = {} #crea un dizionario per memorizzare la PMI di ciascun bigramma
    for bigramma,frequenza in bigrammi_frequenza: #si scorre la lista "bigrammi_frequenza"
        frequenza_parola1 = tokens.count(bigramma[0]) #si ottiene la frequenza della prima parola del bigramma (che è una tupla (token,token))
        frequenza_parola2 = tokens.count(bigramma[1]) #si ottiene la frequenza della seconda parola del bigramma
        PMI_bigramma = round(math.log2((frequenza*len(tokens1))/(frequenza_parola1*frequenza_parola2)),3) #calcola la PMI del bigramma (NB: tokens1 è una variabile globale)
        bigrammi_PMI.update({bigramma:PMI_bigramma}) #aggiorna il dizionario con il nuovo valore {bigramma:PMI_bigramma}
    bigrammi_PMI_decrescente = sorted(bigrammi_PMI.items(), key=lambda x:x[1], reverse=True) #a partire dal dizionario di ottiene una lista ordinata per valori decrescenti con la funzione di libreria "sorted"
    return bigrammi_PMI_decrescente[:20] #si stampano i primi 20 elementi della lista con lo slicing

print("I 20 bigrammi Aggettivo, Sostantivo con Pointwise Mutual Information massima (e relativo valore)")
stampa_lista_di_tuple(trova_bigrammi_PMI_massima(estrai_bigrammi_agg_sost(tokens1),tokens1))

I 20 bigrammi Aggettivo, Sostantivo con Pointwise Mutual Information massima (e relativo valore)
('generous', 'friendship')________________________12.737
('various', 'illnesses')__________________________12.737
('tenderer', 'recollection')______________________12.737
('intellectual', 'solitude')______________________12.737
('actual', 'disparity')___________________________12.737
('daily', 'reach')________________________________12.737
('separate', 'lawn')______________________________12.737
('mutual', 'connexions')__________________________12.737
('beautiful', 'moonlight')________________________12.737
('solemn', 'nonsense')____________________________12.737
('worthy', 'employment')__________________________12.737
('limited', 'remnant')____________________________12.737
('earliest', 'patents')___________________________12.737
('endless', 'creations')__________________________12.737
('successive', 'parliaments')_____________________12.737
('Principal', 'seat')___________________________

In [8]:
def ottieni_frasi_tokenizzate(frasi): #funzione ausiliaria che restituisce un dizionario di frasi tokenizzate
    frasi_tokenizzate = {}
    for frase in frasi: #si scorre la lista di frasi in input
        tokens_frase = nltk.tokenize.word_tokenize(frase) #tokenizza ciascuna frase
        frasi_tokenizzate.update({frase:tokens_frase}) #il dizionario è aggiornato con la nuova coppia {chiave:valore}
    return frasi_tokenizzate #restituisce un dizionario in cui le chiavi sono le frasi originarie e i valori sono le frasi tokenizzate individualmente

def trova_hapax(tokens): #funzione ausiliaria che restituisce in output la lista di hapax
    hapax = [token for token in list(set(tokens)) if tokens.count(token) == 1] #mediante la sintassi di list comprehension si estraggono i token che ricorrono solo 1 volta; si usa list(set(tokens)) per ottenere il vocabolario di parole tipo
    return hapax

def seleziona_frasi_10_20(corpus): #la funzione seleziona le frasi con lunghezza compresa tra 10 e 20 token, di cui la metà NON sono hapax
    frasi,tokens = splitter_e_tokenizzatore(corpus) #estrae frasi e tokens dal corpus
    frasi_tokenizzate = ottieni_frasi_tokenizzate(frasi) #ottiene il dizionario di frasi tokenizzate
    hapax = trova_hapax(tokens) #ottiene la lista di hapax del corpus
    frasi_selezionate = []
    for frase,tokens_frase in frasi_tokenizzate.items(): #si scorre il dizionario
        if len(tokens_frase)>=10 and len(tokens_frase)<=20: #vengono prese in considerazione solo le frasi la cui lunghezza è compresa tra 10 e 20 token
            i = 0 #si inizializza a 0 una variabile i (contatore) che tiene il conto dei token che NON sono hapax
            for token in tokens_frase: #si scorre la lista di tokens (di ciascuna frase)
                if token not in hapax: #se il token NON è presente nella lista di hapax il contatore si aggiorna
                    i += 1
            if i >= math.floor(len(tokens_frase)/2): #se il valore del contatore è almeno pari alla metà della lunghezza della frase allora la frase viene aggiunta alla lista "frasi_selezionate"
                frasi_selezionate.append(frase)
    return frasi_selezionate #restituisce la lista contenente le frasi selezionate

In [9]:
def media_distrFreq_max_min(frasi,tokens): #la funzione calcola la media della distribuzione di frequenza dei token per ogni frase della lista in input e restituisce le frasi con il valore massimo e minimo
    frasi_tokenizzate = ottieni_frasi_tokenizzate(frasi) #si ottiene il dizionario di frasi tokenizzate
    max = 0 #si inizializza a 0 il valore massimo
    min = float("inf") #si inizializza a infinito il valore minimo
    for frase,tokens_frase in frasi_tokenizzate.items(): #si scorre il dizionario
        somma = 0 #si inizializza a 0 la variabile che contiene la somma
        for token in tokens_frase: #si scorre la lista di tokens (per ciascuna frase)
            frequenza_token = tokens.count(token) #si ottiene la frequenza del token nell'intero corpus
            somma += frequenza_token #somma le frequenze di ciascun token della frase
        media_distrFreq_frase = round(somma/len(tokens_frase),3) #calcola la media della distribuzione di frequenza dei token per ogni frase, ovvero: si divide il totale della frequenza di tutti i token per il numero di token (il valore è arrotondato a 3 cifre decimali)
        
        if media_distrFreq_frase > max: #se il valore è maggiore del valore massimo, quest'ultimo viene aggiornato
            max = media_distrFreq_frase
            frase_max = frase #memorizza la frase con il valore massimo

        if media_distrFreq_frase < min: #se il valore è minore del valore minimo, quest'ultimo viene aggiornato
            min = media_distrFreq_frase
            frase_min = frase #memorizza la frase con il valore minimo

    return frase_max, max, frase_min, min

frase_max1,max1,frase_min1,min1 = media_distrFreq_max_min(seleziona_frasi_10_20(corpus1),tokens1)
print(f"La frase con la media della distribuzione di frequenza dei token più alta, ovvero {max1}, è:\n\t{frase_max1}")
print(f"La frase con la media della distribuzione di frequenza dei token più bassa, ovvero {min1}, è:\n\t{frase_min1}")

La frase con la media della distribuzione di frequenza dei token più alta, ovvero 133.909, è:
	"No, papa, nobody thought of your walking.
La frase con la media della distribuzione di frequenza dei token più bassa, ovvero 26.3, è:
	A worthy employment for a young lady's mind!


In [10]:
def markov2_max(frasi,tokens): #la funzione calcola la probabilità delle frasi secondo un modello di Markov di ordine 2 e restituisce quella con probabilità massima 
    frasi_tokenizzate = ottieni_frasi_tokenizzate(frasi) #si ottiene il dizionario di frasi tokenizzate
    bigrammi_corpus = list(nltk.bigrams(tokens)) #estrae i bigrammi di token dall'intero corpus
    trigrammi_corpus = list(nltk.trigrams(tokens)) #estrae i trigrammi di token dall'intero corpus
    max = 0 #si inizializza a 0 il valore massimo

    for frase,tokens_frase in frasi_tokenizzate.items(): #si scorre il dizionario
        bigrammi_frase = list(nltk.bigrams(tokens_frase)) #estrae i bigrammi di token dalla frase
        trigrammi_frase = list(nltk.trigrams(tokens_frase)) #estrae i trigrammi di token dalla frase

        prob_parola1 = tokens.count(tokens_frase[0])/len(tokens) #si calcola la probabilità della prima parola della catena di Markov
        prob_parola2 = (bigrammi_corpus.count(bigrammi_frase[0]))/tokens.count(tokens_frase[0]) #si calcola la probabilità della seconda parola della catena di Markov
        prodotto = prob_parola1 * prob_parola2 #si inizializza una variabile che dovrà contenere il prodotto complessivo della catena di Markov
        
        for i in range(len(trigrammi_frase)): #si scorrono i trigrammi della frase
            frequenza_trigramma = trigrammi_corpus.count(trigrammi_frase[i]) #estrae la frequenza del trigramma
            frequenza_bigramma = bigrammi_corpus.count(bigrammi_frase[i]) #estrae la frequenza del bigramma
            prob_condizionata_parola = frequenza_trigramma/frequenza_bigramma #calcola la probabilità condizionata della seguente parola (dalla terza in poi) nella catena di Markov
            prodotto *= prob_condizionata_parola #il valore è moltiplicato al prodotto complessivo
        
        if prodotto > max: #se il prodotto complessivo è maggiore del valore massimo, quest'ultimo si aggiorna
            max = prodotto
            frase_max = frase #si memorizza la frase corrispondete al prodotto massimo
            
    return frase_max, max

frase_max_markov, max_markov = markov2_max(seleziona_frasi_10_20(corpus1),tokens1)
print(f"La frase con la probabilità più alta secondo il modello di Markov di ordine 2, ovvero {max_markov}, è:\n\t{frase_max_markov}")

La frase con la probabilità più alta secondo il modello di Markov di ordine 2, ovvero 7.323861139592795e-05, è:
	I only doubt whether he will ever take us anywhere else.


In [11]:
def ottieni_NE(tokens_PoS): #la funzione ottiene un dizionario di NE a partire dalla lista di tuple (token,PoS)
    albero_NE = nltk.ne_chunk(tokens_PoS) #si genera l'albero delle Named Entities a partire dalla lista di tuple (token,PoS)
    NE = {} #viene creato un dizionario vuoto
    for nodo in albero_NE: #si scorre l'albero attraverso i nodi
        if hasattr(nodo, 'label'): #se il nodo ha attributo 'label' si accede all'attributo label del nodo, che contiene la classe dell'entità
            classe = nodo.label() #memorizza il nome della classe
            entita = " ".join([token for token,PoS in nodo.leaves()]) #ricostruisce la stringa completa dell'entità
            if classe in NE.keys(): #se "classe" è tra le chiavi del dizionario NE allora la lista di entità corrispondenti viene aggiornata con il nuovo elemento
                NE[classe].append(entita)
            else: #altrimenti "classe" diventa una nuova chiave del dizionario
                NE[classe] = [entita]
    return NE #restituisce un dizionario in cui le chiavi sono le classi di NE e i valori sono liste contenenti i tokens appartenenti alla classe

def trova_NE_frequenti(NE,classe): #la funzione restituisce gli elementi più frequenti (con relativa frequenza) per una classe di NE
    FreqDist_classe_NE = FreqDist(NE[classe]).most_common(15)
    return FreqDist_classe_NE #restituisce la lista delle prime 15 tuple (entità,frequenza) ordinate per frequenza decrescente

for classe in ottieni_NE(tokens_PoS1).keys(): #si scorrono le chiavi del dizionario NE e per ogni classe di NE si stampano le entità più frequenti trovate
    print(f"{classe}:")
    stampa_lista_di_tuple(trova_NE_frequenti(ottieni_NE(tokens_PoS1),classe))
    print("")

PERSON:
Miss Taylor_______________________________________13
Mr. Knightley_____________________________________13
Elizabeth_________________________________________13
Emma______________________________________________12
Mr. Weston________________________________________12
Sir Walter________________________________________12
Mr. Woodhouse_____________________________________6
Lady Elliot_______________________________________6
Anne______________________________________________5
Mary______________________________________________5
Lady Russell______________________________________5
James_____________________________________________4
Mr. Elton_________________________________________4
Sir Walter Elliot_________________________________4
Hartfield_________________________________________3

GPE:
London____________________________________________4
Emma______________________________________________3
Weston____________________________________________2
Isabella____________________________________